In [1]:
from solve_LMI import solve_LMI
from solve_LP import solve_LP
import numpy as np
from scipy.linalg import toeplitz
from circulant_embedding import faster_A
from scipy.sparse import diags
from SCGAL import run_solver
import time
import json

In [2]:
n = 1
max_runs = 100

starting_d = 21
ending_d = 25
ds = list(range(starting_d, ending_d+1))
circ_emb_obj = np.zeros((max_runs, len(ds)))
circ_emb_time = np.zeros((max_runs, len(ds)))
LMI_obj = np.zeros((max_runs, len(ds)))
LMI_time = np.zeros((max_runs, len(ds)))
SCGAL_obj = np.zeros((max_runs, len(ds)))
SCGAL_time = np.zeros((max_runs, len(ds)))
cs = np.random.normal(size=(100, 4*ds[-1], len(ds)))

# d = ds[0]
# c = cs[:4*d, 0]

In [3]:
# Circulant embedding
for i in range(len(ds)):
    d = ds[i]

    A = faster_A(n, d)
    A_0 = np.ones(np.shape(A)[0])

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        c = cs[num_iterations, :4*d, i]
        tic = time.perf_counter()
        status_LP, optimal_value_LP, optimal_vars_LP = solve_LP(A_0, A, c, verbose=False)
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        circ_emb_obj[num_iterations, i] = optimal_value_LP
        circ_emb_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)
        
        # print("LP Status:", status_LP)
        # print("LP Optimal Value:", optimal_value_LP)
        # print("LP Optimal Variables:", [var.value for var in optimal_vars_LP])

21 0.06898662500316277
21 0.10647731000790372
21 0.13682017102837563
21 0.17107174306875095
21 0.20327099005226046
21 0.23572100908495486
21 0.2666808510548435
21 0.30352857103571296
21 0.3384672670508735
21 0.3847203040495515
21 0.4184270650148392
21 0.4536684440099634
21 0.4861380899674259
21 0.5179294589906931
21 0.552079604996834
21 0.5836332309991121
21 0.6166446969727986
21 0.6500672799884342
21 0.6803751939442009
21 0.7066952929017134
21 0.7355654308921658
21 0.7698768608970568
21 0.8018932428676635
21 0.8297572188312188
21 0.8599853828200139
21 0.8903469098149799
21 0.9203801218536682
21 0.954190713877324
21 1.002905057859607
21 1.0441856638644822
21 1.0789810618734919
21 1.1097179689095356
21 1.138750227924902
21 1.1698938509216532
21 1.2052783369435929
21 1.2396799179259688
21 1.27137856092304
21 1.3024622938828543
21 1.3317170008667745
21 1.361928409896791
21 1.3977606389089487
21 1.4290310039068572
21 1.4613072408828884
21 1.4976103659137152
21 1.5357534979120828
21 1.56741

In [4]:
# LMI formulation with toeplitz constraints
for i in range(len(ds)):
    d = ds[i]

    col = np.zeros(2*d + 1, dtype=complex)
    col[0] = 1
    A_0 = toeplitz(col)

    A = []

    for m in range(1, 2*d + 1):
        col = np.zeros(2*d + 1, dtype=complex)
        col[m] = 1
        mx = toeplitz(col)
        A.append(mx)

    for m in range(1, 2*d + 1):
        col = np.zeros(2*d + 1, dtype=complex)
        col[m] = -1j
        mx = toeplitz(col)
        A.append(mx)

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        c = cs[num_iterations, :4*d, i]
        tic = time.perf_counter()
        status_LMI, optimal_value_LMI, optimal_vars_LMI = solve_LMI(A_0, A, c, verbose=False)
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        LMI_obj[num_iterations, i] = optimal_value_LMI
        LMI_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)

    # print("Toeplitz LMI Status:", status_LMI)
    # print("Toeplitz LMI Optimal Value:", optimal_value_LMI)
    # print("Toeplitz LMI Optimal Variables:", [var.value for var in optimal_vars_LMI])

21 0.24396026699105278
21 0.4344492870150134
21 0.7657738240086474
21 0.997328773024492
21 1.2322213080478832
21 1.513688430073671
21 1.7710641800658777
21 2.06425867107464
21 2.9258857720997185
21 3.121500638080761
21 3.3626619790447876
21 3.5340512540424243
21 3.855833281064406
21 5.3502837040578015
21 5.985690019093454
21 6.7529495230992325
21 7.018457108060829
21 7.725962594035082
21 7.982288322004024
21 8.269803136994597
21 8.64663458103314
21 8.9506405920838
21 9.119670479092747
21 9.313040820125025
21 9.624490575108211
21 9.92913187108934
21 10.195452232088428
21 10.453685315093026
21 10.82380949013168
21 11.023228978156112
21 11.34838203014806
21 15.359012789151166
21 15.788540408131666
21 16.12572285917122
21 17.406658255204093
21 17.591123524180148
21 17.99238912120927
21 18.330346614238806
21 19.01089526427677
21 19.311702514241915
21 21.3384137332323
21 21.69737819826696
21 21.979288964241277
21 22.413898445200175
21 23.06335858919192
21 23.499859939154703
21 23.66468422516

In [5]:
# SPARSE LMI formulation for Sketchy CGAL
for i in range(len(ds)):
    d = ds[i]

    diag = np.ones(2*d + 1, dtype=complex)
    A_0 = diags(diag)

    A = []

    for m in range(1, 2*d + 1):
        diag = np.ones(2*d + 1 - m, dtype=complex)
        mx = diags([diag, diag],offsets=[-m,m])
        A.append(mx)

    for m in range(1, 2*d + 1):
        diag = np.ones(2*d + 1 - m, dtype=complex)
        mx = diags([-1j*diag, 1j*diag],offsets=[-m,m])
        A.append(mx)

    A_norm = 2*d

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        c = cs[num_iterations, :4*d, i]
        tic = time.perf_counter()
        U, Lambda, objective, Omega, z, y, S = run_solver(A_0, A, c, 2*d + 1, 4*d,
                                                        alpha=2*np.linalg.norm(c,ord=2), A_norm=A_norm,
                                                        R=1, T=5000,
                                                        trace_mode='bound')
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        SCGAL_obj[num_iterations, i] = -objective
        SCGAL_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)

    # print("Calculated Objective:", -objective)

21 28.493672015029006


ArpackNoConvergence: ARPACK error -1: No convergence (321 iterations, 0/1 eigenvectors converged)

In [6]:
my_dict = {'ds': list(ds), 'circ_emb_obj': circ_emb_obj.tolist(), 'circ_emb_time': circ_emb_time.tolist(),
           'LMI_obj': LMI_obj.tolist(), 'LMI_time': LMI_time.tolist(),
           'SCGAL_obj': SCGAL_obj.tolist(), 'SCGAL_time': SCGAL_time.tolist()}
# Save the results to a JSON file
file_name = 'ds' + str(ds[0]) + 'to' + str(ds[-1]) + '.json'
with open(file_name, 'w') as f:
    json.dump(my_dict, f)

In [7]:
with open(file_name, 'r') as f:
    loaded_dict = json.load(f)

In [8]:
ds_loaded = np.array(loaded_dict['ds'])
circ_emb_obj_loaded = np.array(loaded_dict['circ_emb_obj'])
circ_emb_time_loaded = np.array(loaded_dict['circ_emb_time'])
LMI_obj_loaded = np.array(loaded_dict['LMI_obj'])
LMI_time_loaded = np.array(loaded_dict['LMI_time'])
SCGAL_obj_loaded = np.array(loaded_dict['SCGAL_obj'])
SCGAL_time_loaded = np.array(loaded_dict['SCGAL_time'])

In [9]:
assert np.allclose(ds, ds_loaded)
assert np.allclose(circ_emb_obj, circ_emb_obj_loaded)
assert np.allclose(circ_emb_time, circ_emb_time_loaded)
assert np.allclose(LMI_obj, LMI_obj_loaded)
assert np.allclose(LMI_time, LMI_time_loaded)
assert np.allclose(SCGAL_obj, SCGAL_obj_loaded)
assert np.allclose(SCGAL_time, SCGAL_time_loaded)

In [10]:
print(circ_emb_obj_loaded[0, 0])
print(LMI_obj_loaded[0, 0])
print(SCGAL_obj_loaded[0, 0])

-16.10347688349664
-17.523393062766306
-16.84000975691535


In [ ]:
# Updated code concatenate data
start_idx = [1, 6, 11, 16, 21]
end_idx = [5, 10, 15, 20, 25]

for i in range(len(start_idx)):
    file_name = 'ds' + str(start_idx[i]) + 'to' + str(end_idx[i]) + '.json'

    with open(file_name, 'r') as f:
        loaded_dict = json.load(f)

    if i == 0:
        ds_loaded = np.array(loaded_dict['ds'])
        circ_emb_obj_loaded = np.array(loaded_dict['circ_emb_obj'])
        circ_emb_time_loaded = np.array(loaded_dict['circ_emb_time'])
        LMI_obj_loaded = np.array(loaded_dict['LMI_obj'])
        LMI_time_loaded = np.array(loaded_dict['LMI_time'])
        SCGAL_obj_loaded = np.array(loaded_dict['SCGAL_obj'])
        SCGAL_time_loaded = np.array(loaded_dict['SCGAL_time'])
    else:
        ds_loaded = np.concatenate((ds_loaded, np.array(loaded_dict['ds'])), axis=0)
        circ_emb_obj_loaded = np.concatenate((circ_emb_obj_loaded, np.array(loaded_dict['circ_emb_obj'])), axis=1)
        circ_emb_time_loaded = np.concatenate((circ_emb_time_loaded, np.array(loaded_dict['circ_emb_time'])), axis=1)
        LMI_obj_loaded = np.concatenate((LMI_obj_loaded, np.array(loaded_dict['LMI_obj'])), axis=1)
        LMI_time_loaded = np.concatenate((LMI_time_loaded, np.array(loaded_dict['LMI_time'])), axis=1)
        SCGAL_obj_loaded = np.concatenate((SCGAL_obj_loaded, np.array(loaded_dict['SCGAL_obj'])), axis=1)
        SCGAL_time_loaded = np.concatenate((SCGAL_time_loaded, np.array(loaded_dict['SCGAL_time'])), axis=1)


final_index = 21
ds_loaded = ds_loaded[:final_index]
circ_emb_obj_loaded = circ_emb_obj_loaded[:, :final_index]
circ_emb_time_loaded = circ_emb_time_loaded[:, :final_index]
LMI_obj_loaded = LMI_obj_loaded[:, :final_index]
LMI_time_loaded = LMI_time_loaded[:, :final_index]
SCGAL_obj_loaded = SCGAL_obj_loaded[:, :final_index]
SCGAL_time_loaded = SCGAL_time_loaded[:, :final_index]
    